In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

In [ ]:

from prepare_input import prepare_input_dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix

import tensorflow as tf
# Enable Eager Execution
tf.enable_eager_execution() #kind of useless for the estimator

# Set Verbosity
tf.logging.set_verbosity(tf.logging.ERROR)

BLOCK_SIZE = 100


# Load training and eval data
if __name__ == "__main__":

    data_dir = 'Training_Out'
    #seed = 5
    num_experiments = 20
    num_epochs = 1  # Number of times we go over all the samples in train_data


    for seed in range(num_experiments):
        ((train_data, train_labels, train_times), (eval_data, eval_labels, eval_times)) = prepare_input_dataset(seed, data_dir)
        print(train_data.shape, train_labels.shape)
        print(eval_data.shape, eval_labels.shape)

        checkpoint_path = "lstm_model_training/cp-{experiment:04d}.ckpt"
        #checkpoint_path = "lstm_model_training/cp-{experiment:04d}-{epoch:04d}.ckpt"
        #checkpoint_dir = os.path.dirname(checkpoint_path)


        inputs = tf.keras.Input(shape=(None,2) )
        #lstm_1 = tf.keras.layers.LSTM(32, return_sequences=True)(inputs)
        lstm = tf.keras.layers.LSTM(256)(inputs)
        outputs = tf.keras.layers.Dense(3, activation='softmax')(lstm)

        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        # Specify the training configuration (optimizer, loss, metrics)
        model.compile(optimizer=tf.keras.optimizers.SGD(0.01), #learning rate 0.01  # Optimizer
                    # Loss function to minimize
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                    # List of metrics to monitor
                    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

        model.summary()
        #model.save_weights(checkpoint_path.format(experiment=seed, epoch=0))
        model.save_weights(checkpoint_path.format(experiment=seed))


        ttrue = []
        tpred = []
        for e in range(num_epochs):
            ytrue = []
            ypred = []
            for n, i in enumerate(np.random.permutation(train_data.shape[0])):

                #model.load_weights(checkpoint_path.format(experiment=seed, epoch=e))
                model.load_weights(checkpoint_path.format(experiment=seed))
                # Train the model
                batch=100 #alternative to num_epochs, repeat sample batch number of times
                x = np.repeat(np.expand_dims(train_data[i], axis=0), batch, axis=0)
                y = np.repeat(train_labels[i:i+1], batch, axis=0)
                print("Exp %d epoch %d -- %d / %d " % (seed, e+1, n+1, train_data.shape[0]))
                history = model.fit(x[:, :500, :], y,   # use [:, :500, :] to take first 500 steps in the sequence, since it's too big.
                        batch_size=batch,
                        epochs=1,
                        verbose=0)
                #model.save_weights(checkpoint_path.format(experiment=seed, epoch=e+1))
                model.save_weights(checkpoint_path.format(experiment=seed))
                #print('# save model')

            for i in range(train_data.shape[0]):

                predictions = model.predict(np.expand_dims(train_data[i], axis=0)[:, :500, :])
                ypred.append(np.argmax(predictions, 1)[0])
                ytrue.append(train_labels[i:i+1])


            # The returned "history" object holds a record
            # of the loss values and metric values during training
            #print('\nhistory dict:', history.history)
            print("Epoch %d / %d  accuracy: " % (e+1, num_epochs), end = '')
            print(accuracy_score(np.asarray(ytrue),np.asarray(ypred)))
            print(confusion_matrix(np.asarray(ytrue),np.asarray(ypred)))
            ttrue = ttrue + ytrue
            tpred = tpred + ypred

        #At the end of training for num_epochs
        print("Accumulated accuracy: ")
        print(accuracy_score(np.asarray(ttrue),np.asarray(tpred)))
        print(confusion_matrix(np.asarray(ttrue),np.asarray(tpred)))


        test_true = []
        test_pred = []
        #model.load_weights(checkpoint_path.format(experiment=seed, epoch=num_epochs))
        model.load_weights(checkpoint_path.format(experiment=seed))
        for i in range(eval_data.shape[0]):

            # Evaluate the model on the test data using `evaluate`
            #print('\n# Evaluate on test data')
            #results = model.evaluate(np.expand_dims(eval_data[i], axis=0), eval_labels[i:i+1], batch_size=1)
            #print('test loss, test acc:', results)

            predictions = model.predict(np.expand_dims(eval_data[i], axis=0)[:, :500, :])
            test_pred.append(np.argmax(predictions, 1)[0])
            test_true.append(eval_labels[i:i+1])

        print("Evaluation accuracy: ")
        print(accuracy_score(np.asarray(test_true),np.asarray(test_pred)))
        print(confusion_matrix(np.asarray(test_true),np.asarray(test_pred)))